1. Import the dependencies

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

2. Load data

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = torchvision.datasets.MNIST(root='./data', train = True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True)

100%|██████████| 9.91M/9.91M [00:00<00:00, 11.4MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 340kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 3.17MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.88MB/s]


3. CNN Model

In [ ]:
class CNN_mnist(nn.Module):
  def __init__(self):
    super(CNN_mnist, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1) # Output 16*28*28
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2) # Output 16*14*14
    self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1) #Output 32*14*14
    self.fc1 = nn.Linear(32*7*7, 128)
    self.fc2 = nn.Linear(128, 10) # 10 multiclassification 0-9

  def forward(self, x):
    x = F.relu(self.conv1(x)) # conv1 => relu => pooling
    x = self.pool(x)
    x = F.relu(self.conv2(x)) # conv2 => relu => pooling
    x = self.pool(x)
    x = x.view(-1, 32*7*7) # flatten
    x = F.relu(self.fc1(x)) # Fully connected + relu
    x = self.fc2(x) # output layer
    return x

Initialize the model

In [ ]:
device = torch.device('cude' if torch.cuda.is_available() else 'cpu')
print(device)
model = CNN_mnist().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# train loop
num_epochs = 5 # 5 times iterartion => epoch
for epoch in range(num_epochs):
  for i,(images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    loss = criterion(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

cpu
Epoch [1/5], Loss: 2.3171
Epoch [1/5], Loss: 2.2924
Epoch [1/5], Loss: 2.2880
Epoch [1/5], Loss: 2.2723
Epoch [1/5], Loss: 2.2624
Epoch [1/5], Loss: 2.2564
Epoch [1/5], Loss: 2.2230
Epoch [1/5], Loss: 2.1945
Epoch [1/5], Loss: 2.2183
Epoch [1/5], Loss: 2.1478
Epoch [1/5], Loss: 2.1036
Epoch [1/5], Loss: 2.0415
Epoch [1/5], Loss: 2.1007
Epoch [1/5], Loss: 1.9825
Epoch [1/5], Loss: 1.9794
Epoch [1/5], Loss: 1.8637
Epoch [1/5], Loss: 1.8962
Epoch [1/5], Loss: 1.7986
Epoch [1/5], Loss: 1.7098
Epoch [1/5], Loss: 1.6301
Epoch [1/5], Loss: 1.6484
Epoch [1/5], Loss: 1.4868
Epoch [1/5], Loss: 1.5669
Epoch [1/5], Loss: 1.4822
Epoch [1/5], Loss: 1.4930
Epoch [1/5], Loss: 1.2891
Epoch [1/5], Loss: 1.2411
Epoch [1/5], Loss: 1.2347
Epoch [1/5], Loss: 1.2528
Epoch [1/5], Loss: 1.1363
Epoch [1/5], Loss: 1.0027
Epoch [1/5], Loss: 0.8081
Epoch [1/5], Loss: 1.0019
Epoch [1/5], Loss: 0.9054
Epoch [1/5], Loss: 0.7389
Epoch [1/5], Loss: 0.9235
Epoch [1/5], Loss: 0.7886
Epoch [1/5], Loss: 0.6854
Epoch [1

Test Accuracy

In [ ]:
correct = 0
total = 0
with torch.no_grad():
  for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print(f'Test Accuracy: {100 * correct / total}%')

Test Accuracy: 99.0%
